## 基于NGAWest2数据库的选波程序
开发者：Vincent  
日期：2024年2月4日

In [1]:
%reset -f

### 1.实例化选波器

In [2]:
from Selecting import Selecting
selector = Selecting()

### 2.导入地震动数据库文件

In [3]:
# 文件路径
file_acc = r'F:\NGAWest2\Acceleration.hdf5'
file_vel = r'F:\NGAWest2\Velocity.hdf5'
file_disp = r'F:\NGAWest2\Displacement.hdf5'
file_spec = r'F:\NGAWest2\Spectra.hdf5'
file_info = r'F:\NGAWest2\Info.hdf5'
# 导入文件
selector.import_files(file_acc, file_vel, file_disp, file_spec, file_info)

正在校验文件 - F:\NGAWest2\Acceleration.hdf5
正在校验文件 - F:\NGAWest2\Velocity.hdf5
正在校验文件 - F:\NGAWest2\Displacement.hdf5
正在校验文件 - F:\NGAWest2\Spectra.hdf5
正在校验文件 - F:\NGAWest2\Info.hdf5


### 3.导入反应谱
注：反应谱文件应为包含两列数据的文本文档，第一列为周期，第二列为加速度反应谱，单位应为"g"，且反应谱应包含T=0时的谱值

In [ ]:
selector.target_spectra(r'F:\MyProgram\GroundMotions\DBE谱.txt')

### 4.定义地震动缩放方法
Args:  
approach (str): 缩放方法  
* [a] 按Sa(0)(即PGA)匹配反应谱, para=`None` or Sa  
* [b] 按Sa(Ta)匹配反应谱, para=Ta or (Ta, Sa)  
* [c] 按Sa(Ta~Tb)匹配反应谱(几何平均数), 最小化RMSE, para=(Ta, Tb)  
* [d] 按Sa,avg(Ta~Tb)匹配反应谱, para=(Ta, Tb) or (Ta, Tb, Sa)  
* [e] 指定缩放系数, para=SF  

para (float | tuple): 缩放参数，与`approach`的取值有关

注：  
(1)原始地震动的反应谱需按照给定的缩放方法进行缩放后才能与目标谱进行对比筛选，如《抗规》中根据PGA进行地震动的缩放；  
(2)当`approach`为"a"、"b"或"d"时，`para`可额外指定加速度谱值，如`scaling_approach("a")`表示将反应谱的PGA缩放至与目标谱一致，`scaling_approach("a", para=0.4)`表示将反应谱的PGA缩放至0.4g。

In [ ]:
selector.scaling_approach('a')

### 5.定义反应谱匹配方法

Args:
rules (list[str]): 匹配规则
* [full] 按照给定的反应谱的最大周期范围完全匹配(归一化军方法误差NRSME最小), para=[None]
* [a] 按Sa(0)(即PGA)匹配反应谱, para=[None]  
* [b] 按Sa(Ta)匹配反应谱, para=[Ta]  
* [c] 按Sa(Ta~Tb)匹配反应谱(几何平均数), 最小化RMSE, para=[(Ta, Tb)]  
* [d] 按Sa,avg(Ta~Tb)匹配反应谱, para=[(Ta, Tb)]  

para (list[float | tuple]): 缩放参数，与`approach`的取值有关  
weight (list[tuple]): 各匹配规则的权重系数

Example:
`matching_rules(rules=['full', 'b', 'c'], para=[None, 1.5, (1, 2)], weight=[1, 1.2, 0.8])`表示在匹配反应谱（计算拟合误差）时综合考虑全周期范围、Sa(1.5)处谱值和Sa(1~2)范围的匹配程度，
对应的权重依次为1、1.2、和0.8

注:  
(1)`scaling_approach`方法中选用的`approach`参数不得作为该方法的`rules`参数
（这表示之前定义的缩放方法不得用于判断反应谱匹配程度）；  
(2)可以定义多个相同的匹配规则，如`matching_rules(rules=['b', 'b'], para=[1.5, 3]`表示在周期为1.5s和3s的周期点进行匹配，两个周期点权重相同。

In [ ]:
selector.matching_rules(rules=['full', 'b', 'b'], para=[None, 4, 2], weight=[7, 3, 3])

### 6.定义约束条件
Args:  
scale_factor (tuple, optional): 缩放系数，默认`None`  
PGA (tuple, optional): PGA，默认`None`  
magnitude (tuple, optional): 震级，默认`None`  
Rjb (tuple, optional): 默认`None`  
Rrup (tuple, optional): 默认`None`  
vs30 (tuple, optional): 剪切波速，默认`None`  
D5_95 (tuple, optional): 有效持时，默认`None`  
strike_slip (str, optional): 振源机制，默认`'all' ` 
* [all] all types
* [a] strike slip
* [b] normal/oblique
* [c] reverse/oblique
* [d] strike slip + normal/oblique
* [e] strike slip + reverse/oblique
* [f] normal/oblique + reverse/oblique  

pulse (str | bool, optional): 脉冲型地震，默认`'all'`
* [all] 不限定范围
* [True] 仅脉冲型
* [False] 仅非脉冲型  

N_events (int, optional): 相同地震事件所允许的最大出现次数，默认`None`   
RSN (tuple, optional): RSN，默认`None`  
component (list, optional): 地震动分量，默认`['H1', 'H2', 'V']`，可根据需要删减列表元素

In [ ]:
selector.constrain_range(N_events=10, scale_factor=(3, 10), component=['H1'])

### 6.进行选波计算
Args:  
number (int): 需要的地震动数量  

Returns:  
list[str]: 包括所有选波结果地震动名（无后缀）的列表  
dict: {地震动名: (缩放系数, 匹配误差)}

In [ ]:
selected_records, records_info = selector.run(10)
# 返回两个变量，其中selected_records包括地震动名，records_info包括缩放系数和匹配误差信息

### 7.提取地震动数据
Args:  
output_dir (str | Path): 输出文件夹路径  
type_ (str): 'A'、'V'或'D'，代表加速度、速度和位移  
RSN (int, optional): 按给定的单个RSN序号提取，默认`None`  
RSN_list (list, optional): 按RSN列表提取，默认`None`  
RSN_range (list, optional): 按RSN范围提取，默认`None`  
files (list, optional): 按地震动文件名提取，默认`[]`  
file_SF_error (dict, optional): 地震文件-缩放系数字典，默认`{}`  
write_unscaled_record (bool, optional): 是否写入未缩放地震动，默认`True`  
write_norm_record (bool, optional): 是否写入归一化地震动，默认`True`  
write_scaled_records (bool, optional): 是否写入缩放后地震动，默认`True`

注：该方法可以单独用于提取地震动数据（不经过上面的选波过程），此时可通过传入`type_`, `RSN`、`RSN_list`和`RSN_range`来提取。当进行选波时，则应将`run`方法返回的两个变量传入。

In [ ]:
selector.extract_records(r'C:\Users\Admin\Desktop\GroungMotions\results', files=selected_records, file_SF_error=records_info)

### 8.完成选波
结果将生成在指定的文件夹，包括地震动数据文本文档、地震动信息.csv和反应谱.csv

## A.其他功能
不断补充中...

### A.1 查看数据库概括

In [ ]:
selector.check_database()